In [1]:
import numpy as np
import os
import time
from utils import *

In [2]:
phi = relu
dev_phi = dev_relu

fmap = 'rf'

In [32]:
ks = [10000]
d = 1000
Ns = [3000]
Nt = 100

In [33]:
testr = []
test = []


for N in Ns:
    for k in ks:
    # Dataset
       
        dy = d // 2
        dx = d - dy

        u = np.random.randn(dx) / np.sqrt(dx)

        X = np.random.randn(N, dx)
        Y = np.random.randn(N, dy)
        Z = np.concatenate((X, Y), axis=1)  # training data

        G = np.sign(X @ u)

        Xa = np.random.randn(N, dx)
        Za = np.concatenate((Xa, Y), axis=1)  # attack data

        Xt = np.random.randn(Nt, dx)
        Yt = np.random.randn(Nt, dy)
        Zt = np.concatenate((Xt, Yt), axis=1)  # test data

        Gt = np.sign(Xt @ u)
    
        # Feature Map
    

        V = np.random.randn(k, d) / np.sqrt(d)
        
        Phi = phi(Z @ V.transpose())
        Phi_a = phi(Za @ V.transpose())
        Phi_t = phi(Zt @ V.transpose())
        
        V_inv =  np.linalg.pinv(V.transpose())
        P = V_inv @ V.transpose()
        # Sigma = V_inv @ V_inv.transpose()

    
        # Solution
        # K_r = Phi_1 @ Phi_1.transpose()
        K_inv = np.linalg.inv(Phi @ Phi.transpose())
        pPhi = Phi.transpose() @ K_inv

        # Sigma = np.linalg.inv(d * k / 2 * np.eye(d * k) + Phi_1.transpose() @ Phi_1)
        eps = np.sqrt(d) / (100 * k) * V @ np.random.randn(d)  # np.random.multivariate_normal(np.zeros(k), Sigma)  #  * np.sqrt(d / N)
        
        theta = np.linalg.pinv(Phi) @ G
        theta_1 = 0.5 * V @ Z.transpose() @ K_inv @ G + eps
        theta_2 = P @ theta + eps
    
        # Scores
    
        score_attack = 0
        for i in range(N):
            if np.inner(Phi_a[i], theta) * G[i] > 0:
                score_attack += 1
        score_attack /= N
    
        score_test = 0
        for i in range(Nt):
            if np.inner(Phi_t[i], theta) * Gt[i] > 0:
                score_test += 1
        score_test /= Nt
    
        score_train = 0  # sanity check
        for i in range(N):
            if np.inner(Phi[i], theta) * G[i] > 0:
                score_train += 1
        score_train /= N
    
    
        # Scores 1
    
        score_attack_1 = 0
        for i in range(N):
            if np.inner(Phi_a[i], theta_1) * G[i] > 0:
                score_attack_1 += 1
        score_attack_1 /= N
    
        score_test_1 = 0
        for i in range(Nt):
            if np.inner(Phi_t[i], theta_1) * Gt[i] > 0:
                score_test_1 += 1
        score_test_1 /= Nt
    
        score_train_1 = 0
        for i in range(N):
            if np.inner(Phi[i], theta_1) * G[i] > 0:
                score_train_1 += 1
        score_train_1 /= N
    
    
        # Scores 2
    
        score_attack_2 = 0
        for i in range(N):
            if np.inner(Phi_a[i], theta_2) * G[i] > 0:
                score_attack_2 += 1
        score_attack_2 /= N
    
        score_test_2 = 0
        for i in range(Nt):
            if np.inner(Phi_t[i], theta_2) * Gt[i] > 0:
                score_test_2 += 1
        score_test_2 /= Nt
    
        score_train_2 = 0
        for i in range(N):
            if np.inner(Phi[i], theta_2) * G[i] > 0:
                score_train_2 += 1
        score_train_2 /= N
    
    

In [34]:
# z_t = np.random.randn(d)
y = np.random.randn(dy)
z_t = np.concatenate((np.sqrt(dx) * u, y))
phi_z = phi(V @ z_t)

f = theta.transpose() @ phi_z
f

17.474462199834413

In [71]:
phi = relu
dev_phi = dev_relu

gt = 'sign'

k = 10000
d = 100
N = 1000

# xt = np.random.randn(d)
u = np.random.randn(d) / np.sqrt(d)
X = np.random.randn(N, d) #  + np.outer(np.ones(N), xt) / np.sqrt(d)

In [72]:
if 'sign' in gt:
    G = np.sign(X @ u)
    xt = np.sqrt(d) * X.transpose() @ G / np.linalg.norm(X.transpose() @ G)
    Gt = np.sign(np.inner(xt, u)) # np.sign(Xt @ u)
elif 'linear' in gt:
    G = X @ u
    xt = np.sqrt(d) * X.transpose() @ G / np.linalg.norm(X.transpose() @ G)
    Gt = np.inner(xt, u) # Xt @ u
elif 'square' in gt:
    G = (X @ u) ** 2
    xt = np.sqrt(d) * X.transpose() @ G / np.linalg.norm(X.transpose() @ G)
    Gt = (np.inner(xt, u)) ** 2 # (Xt @ u) ** 2

V = np.random.randn(k, d) / np.sqrt(d)
Phi = phi(X @ V.transpose())
phi_t = phi(V @ xt)

V_inv =  np.linalg.pinv(V.transpose())
P = V_inv @ V.transpose()

# ort_test = (np.eye(k) - P) @ Phi_t[0]
    
theta = np.linalg.pinv(Phi) @ G
theta_1 = P @ theta


test_loss = np.linalg.norm(np.inner(phi_t, theta) - Gt) ** 2
train_loss = np.linalg.norm(Phi @ theta - G) ** 2 / N
test_loss_1 = np.linalg.norm(np.inner(phi_t, theta_1) - Gt) ** 2
train_loss_1 = np.linalg.norm(Phi @ theta_1 - G) ** 2 / N

In [74]:
test_loss_1

42.20519857915207

array([[-1.60096694,  0.27174977, -0.21172676, ..., -0.48479041,
        -0.96792269,  1.63459134],
       [ 0.13787489,  0.77991542,  0.25285986, ..., -0.94782649,
         0.8341446 , -0.35706429],
       [-0.47516119,  0.61757114, -0.13649291, ...,  0.07088566,
         1.17678482,  1.75481149],
       ...,
       [-0.59335394, -1.00680831,  0.42989737, ...,  2.33793473,
         0.29115917,  0.47490082],
       [-0.12345399,  0.32266323, -0.12814617, ...,  2.43098824,
         1.74532975,  0.2274105 ],
       [ 1.00606762,  0.67662096,  0.8584355 , ...,  0.44042554,
         1.9993179 ,  0.14236148]])